In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import time

# Data processing

In [ ]:
cleaned_path = r'./1 - Organized data gauge/BRAZIL/DATASETS/BRAZIL_DAILY_1961_2024_CLEANED.h5'

In [3]:
chunk_size = 13000000  # Adjust the chunk size as needed
chunks = []
df_data = pd.DataFrame()

# Read the filtered data in chunks  from the HDF5 file

with pd.HDFStore(cleaned_path, mode='r') as store:
    total_chunks = store.get_storer('table_data_filtered').nrows // chunk_size + 1
    print(f"Rows in table_data_filtered: {store.get_storer('table_data_filtered').nrows}\n")
    start_time, step_time = time.time(), time.time()
    for i, chunk in enumerate(store.select('table_data_filtered', chunksize=chunk_size)):
        if df_data.empty:
            df_data = chunk
        else:
            df_data = pd.concat([df_data, chunk], ignore_index=True)  # Concatenate chunk to df_data_filtered (inplace=True)
        del chunk  # Delete the chunk variable to free up memory
        print(f"Processed chunk {i + 1} of {total_chunks} | Time taken: {(time.time() - step_time):.1f} seconds")
        step_time = time.time()
print(f"Total time taken:  {(time.time() - start_time):.1f} seconds")
df_data

Rows in table_data_filtered: 123611008

Processed chunk 1 of 10 | Time taken: 7.5 seconds
Processed chunk 2 of 10 | Time taken: 10.8 seconds
Processed chunk 3 of 10 | Time taken: 8.9 seconds
Processed chunk 4 of 10 | Time taken: 9.2 seconds
Processed chunk 5 of 10 | Time taken: 11.7 seconds
Processed chunk 6 of 10 | Time taken: 12.8 seconds
Processed chunk 7 of 10 | Time taken: 10.5 seconds
Processed chunk 8 of 10 | Time taken: 15.8 seconds
Processed chunk 9 of 10 | Time taken: 18.2 seconds
Processed chunk 10 of 10 | Time taken: 16.2 seconds
Total time taken:  122.0 seconds


,gauge_code,datetime,rain_mm
0,00047000,1961-01-01,0.0
1,00047000,1961-01-02,0.0
2,00047000,1961-01-03,0.0
3,00047000,1961-01-04,0.0
4,00047000,1961-01-05,0.0
...,...,...,...
123611003,S717,2021-12-27,0.0
123611004,S717,2021-12-28,0.0
123611005,S717,2021-12-29,0.0
123611006,S717,2021-12-30,0.0


In [4]:
del step_time, start_time, total_chunks, chunk_size, store

In [5]:
def calculateQ3(df):

    df_monthly_thresholds = df.copy(deep = True)  # Create a deep copy of the DataFrame
    df_monthly_thresholds['month'] = df_monthly_thresholds['datetime'].dt.month
    df_monthly_thresholds['year'] = df_monthly_thresholds['datetime'].dt.year

    df_monthly_thresholds = df_monthly_thresholds[df_monthly_thresholds['rain_mm'] >= 1.0]  # Filter rainy days first
    df_monthly_thresholds = df_monthly_thresholds.groupby(['gauge_code', 'month'])['rain_mm'].agg(
        Q1=lambda x: x.quantile(0.25) if not x.empty else np.nan,
        Q3=lambda x: x.quantile(0.75) if not x.empty else np.nan
    ).reset_index()

    df_monthly_thresholds['IQR'] = df_monthly_thresholds['Q3'] - df_monthly_thresholds['Q1']
    # df_monthly_thresholds['lower_bound'] = df_monthly_thresholds['Q1'] - 1.5 * df_monthly_thresholds['IQR']
    df_monthly_thresholds['upper_bound'] = df_monthly_thresholds['Q3'] + 1.5 * df_monthly_thresholds['IQR']

    return df_monthly_thresholds

df_monthly_thresholds = calculateQ3(df_data)
df_monthly_thresholds

,gauge_code,month,Q1,Q3,IQR,upper_bound
0,00047000,1,5.60,13.70,8.10,25.850
1,00047000,2,6.20,33.30,27.10,73.950
2,00047000,3,4.65,45.00,40.35,105.525
3,00047000,4,4.60,11.20,6.60,21.100
4,00047000,5,4.60,9.60,5.00,17.100
...,...,...,...,...,...,...
206909,S716,6,3.70,16.10,12.40,34.700
206910,S716,7,1.00,1.00,0.00,1.000
206911,S716,10,7.00,31.00,24.00,67.000
206912,S716,11,5.60,24.00,18.40,51.600


In [6]:
df_data['month'] = df_data['datetime'].dt.month
df_data['year'] = df_data['datetime'].dt.year
df_data

,gauge_code,datetime,rain_mm,month,year
0,00047000,1961-01-01,0.0,1,1961
1,00047000,1961-01-02,0.0,1,1961
2,00047000,1961-01-03,0.0,1,1961
3,00047000,1961-01-04,0.0,1,1961
4,00047000,1961-01-05,0.0,1,1961
...,...,...,...,...,...
123611003,S717,2021-12-27,0.0,12,2021
123611004,S717,2021-12-28,0.0,12,2021
123611005,S717,2021-12-29,0.0,12,2021
123611006,S717,2021-12-30,0.0,12,2021


In [7]:
df_data = df_data.merge(df_monthly_thresholds[['gauge_code', 'month', 'upper_bound']], on=['gauge_code', 'month'], how='outer')
df_data

,gauge_code,datetime,rain_mm,month,year,upper_bound
0,00047000,1961-01-01,0.0,1,1961,25.85
1,00047000,1961-01-02,0.0,1,1961,25.85
2,00047000,1961-01-03,0.0,1,1961,25.85
3,00047000,1961-01-04,0.0,1,1961,25.85
4,00047000,1961-01-05,0.0,1,1961,25.85
...,...,...,...,...,...,...
123611003,S717,2021-12-27,0.0,12,2021,NaN
123611004,S717,2021-12-28,0.0,12,2021,NaN
123611005,S717,2021-12-29,0.0,12,2021,NaN
123611006,S717,2021-12-30,0.0,12,2021,NaN


In [8]:
df_data['upper_bound'] = df_data['upper_bound'].fillna(0)  # Fill NaN values with 0
df_data

,gauge_code,datetime,rain_mm,month,year,upper_bound
0,00047000,1961-01-01,0.0,1,1961,25.85
1,00047000,1961-01-02,0.0,1,1961,25.85
2,00047000,1961-01-03,0.0,1,1961,25.85
3,00047000,1961-01-04,0.0,1,1961,25.85
4,00047000,1961-01-05,0.0,1,1961,25.85
...,...,...,...,...,...,...
123611003,S717,2021-12-27,0.0,12,2021,0.00
123611004,S717,2021-12-28,0.0,12,2021,0.00
123611005,S717,2021-12-29,0.0,12,2021,0.00
123611006,S717,2021-12-30,0.0,12,2021,0.00


In [9]:
df_data.pop('month')
df_data.pop('year')

0            1961
1            1961
2            1961
3            1961
4            1961
             ... 
123611003    2021
123611004    2021
123611005    2021
123611006    2021
123611007    2021
Name: year, Length: 123611008, dtype: int32

In [12]:
df_data['outlier'] = (df_data['rain_mm'] > df_data['upper_bound']).astype(np.uint8)
df_data['year'] = df_data['datetime'].dt.year
df_data

,gauge_code,datetime,rain_mm,upper_bound,outlier,year
0,00047000,1961-01-01,0.0,25.85,0,1961
1,00047000,1961-01-02,0.0,25.85,0,1961
2,00047000,1961-01-03,0.0,25.85,0,1961
3,00047000,1961-01-04,0.0,25.85,0,1961
4,00047000,1961-01-05,0.0,25.85,0,1961
...,...,...,...,...,...,...
123611003,S717,2021-12-27,0.0,0.00,0,2021
123611004,S717,2021-12-28,0.0,0.00,0,2021
123611005,S717,2021-12-29,0.0,0.00,0,2021
123611006,S717,2021-12-30,0.0,0.00,0,2021


In [20]:
df_data = df_data[['gauge_code', 'year', 'outlier']].copy(deep = True)
df_data

,gauge_code,year,outlier
0,00047000,1961,0
1,00047000,1961,0
2,00047000,1961,0
3,00047000,1961,0
4,00047000,1961,0
...,...,...,...
123611003,S717,2021,0
123611004,S717,2021,0
123611005,S717,2021,0
123611006,S717,2021,0


In [22]:
df_data['gauge_code'].nunique()

18503

In [21]:
df_q3_outliers = df_data.groupby(['gauge_code', 'year']).agg(
    count_outliers=('outlier', lambda x: x.sum() if not x.empty else 0),  # Count the number of outliers per gauge_code and year
    active_days=('outlier', 'count')  # Count the number of datetime entries per gauge_code and year
).reset_index()
df_q3_outliers

,gauge_code,year,count_outliers,active_days
0,00047000,1961,7,365
1,00047000,1962,2,365
2,00047000,1963,1,365
3,00047000,1964,1,366
4,00047002,1977,2,23
...,...,...,...,...
346024,S713,2021,2,365
346025,S714,2021,7,365
346026,S715,2021,8,365
346027,S716,2021,4,365


In [23]:
df_q3_outliers['outlier_percentage'] = df_q3_outliers['count_outliers'] / df_q3_outliers['active_days'] * 100  # Calculate the percentage of outliers
df_q3_outliers['q3_outliers'] = 100 - df_q3_outliers['outlier_percentage']  # Calculate the percentage of non-outliers
df_q3_outliers

,gauge_code,year,count_outliers,active_days,outlier_percentage,q3_outliers
0,00047000,1961,7,365,1.917808,98.082192
1,00047000,1962,2,365,0.547945,99.452055
2,00047000,1963,1,365,0.273973,99.726027
3,00047000,1964,1,366,0.273224,99.726776
4,00047002,1977,2,23,8.695652,91.304348
...,...,...,...,...,...,...
346024,S713,2021,2,365,0.547945,99.452055
346025,S714,2021,7,365,1.917808,98.082192
346026,S715,2021,8,365,2.191781,97.808219
346027,S716,2021,4,365,1.095890,98.904110


In [24]:
df_q3_outliers = df_q3_outliers[['gauge_code', 'year', 'q3_outliers']].copy(deep = True)
df_q3_outliers.to_hdf(cleaned_path
                  , key = 'table_q3_outliers'
                  , encoding = 'utf-8'
                  , mode='r+'
                  , append = False
                  , complevel=9
                  , format='table')

In [25]:
df_q3_outliers= pd.read_hdf(cleaned_path, key = 'table_q3_outliers', encoding = 'utf-8')
df_q3_outliers

,gauge_code,year,q3_outliers
0,00047000,1961,98.082192
1,00047000,1962,99.452055
2,00047000,1963,99.726027
3,00047000,1964,99.726776
4,00047002,1977,91.304348
...,...,...,...
346024,S713,2021,99.452055
346025,S714,2021,98.082192
346026,S715,2021,97.808219
346027,S716,2021,98.904110
